# EUROCONTROL R&D ARCHIVE

In [ ]:
import pandas as pd

### AIRAC

In [ ]:
# AIRAC
df_airac = pd.read_csv('./data/202309/AIRAC_2309.csv.gz')
df_airac.head()

### FIR

In [ ]:
df_FIR_2309 = pd.read_csv('./data/202309/FIR_2309.csv.gz')
df_FIR_2309.head()

In [ ]:
df_FIR_2309.describe()

In [ ]:
from ectl_plots_utilities import plot_airspace

print(df_FIR_2309[df_FIR_2309['Airspace ID'].str.startswith('LF')]['Airspace ID'].drop_duplicates())

fig, ax = plot_airspace(df_FIR_2309, airspace_str="LFBBFIR")
fig, ax = plot_airspace(df_FIR_2309, "LFEEFIR", fig, ax)
fig, ax = plot_airspace(df_FIR_2309, "LFFFFIR", fig, ax)
fig, ax = plot_airspace(df_FIR_2309, "LFMMFIR", fig, ax)
fig, ax = plot_airspace(df_FIR_2309, "LFRRFIR", fig, ax)

fig, ax = plot_airspace(df_FIR_2309, airspace_str="LFFFUIR")


## Routes

In [ ]:
# Routes
df_routes = pd.read_csv('./data/202309/Route_2309.csv.gz')
df_routes.head()

In [ ]:
len(df_routes['Route ID'].drop_duplicates())

In [ ]:
from ectl_plots_utilities import plot_route

print(df_routes['Route ID'].drop_duplicates())

plot_route(df_routes[df_routes["Route ID"] == "AKUPO1MLRCL"], route_id="AKUPO1MLRCL")
plot_route(df_routes[df_routes["Route ID"] == "A4"], route_id="A4")
fig = None
ax = None
for r in df_routes[df_routes["Route ID"].str.startswith('VIBOK')]['Route ID'].drop_duplicates():
    fig, ax = plot_route(df_routes[df_routes["Route ID"] == r], fig=fig, ax=ax, route_id=r)
    


## Flights

### Flights

In [ ]:
# Flights
df_flights = pd.read_csv('./data/202309/Flights_20230901_20230930.csv.gz')
df_flights.head()

In [ ]:
df_flights["FILED OFF BLOCK TIME"] = pd.to_datetime(df_flights["FILED OFF BLOCK TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["FILED ARRIVAL TIME"] = pd.to_datetime(df_flights["FILED ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL OFF BLOCK TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL ARRIVAL TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")

In [ ]:
len(df_flights)

In [ ]:
df_flights.iloc[0]

In [ ]:
# Example -- Flights departing from Heathrow
df_flights[df_flights.ADEP=='EGLL']

In [ ]:
## Plot histogram departures and arrivals
from ectl_plots_utilities import plot_histogram_departures_arrivals_planned_actual

# --- Select airport
airport = "EGLL"
plot_histogram_departures_arrivals_planned_actual(df_flights, airport)


### Points

In [ ]:
# Flight Points Filed
df_points_filed = pd.read_csv('./data/202309/Flight_Points_Filed_20230901_20230930.csv.gz')
# Flight Points Actual
df_points_actual = pd.read_csv('./data/202309/Flight_Points_Actual_20230901_20230930.csv.gz')

In [ ]:
# Example of a flight
ectrl_id = 265837588
df_points_filed[df_points_filed["ECTRL ID"] == ectrl_id].head()


In [ ]:
from ectl_plots_utilities import plot_trajectory

# Plot some flights

ectrl_id = 265837588
# Filed
fig, ax = plot_trajectory(df_points_filed[df_points_filed["ECTRL ID"] == ectrl_id], ectrl_id=str(ectrl_id), label="Filed "+str(ectrl_id))
# Actual
fig, ax = plot_trajectory(df_points_actual[df_points_actual["ECTRL ID"] == ectrl_id], ectrl_id=str(ectrl_id), label="Actual "+str(ectrl_id),                           fig=fig, ax=ax, color='red')

ectrl_id = 265837584
# Filed
fig, ax = plot_trajectory(df_points_filed[df_points_filed["ECTRL ID"] == ectrl_id], ectrl_id=str(ectrl_id), label="Filed "+str(ectrl_id))
# Actual
fig, ax = plot_trajectory(df_points_actual[df_points_actual["ECTRL ID"] == ectrl_id], ectrl_id=str(ectrl_id), label="Actual "+str(ectrl_id),                           fig=fig, ax=ax, color='red')


### FIRs

In [ ]:
# Flight FIRS Filed
df_f_firs_filed = pd.read_csv('./data/202309/Flight_FIRs_Filed_20230901_20230930.csv.gz')

# Flight FIRS Actual
df_f_firs_actual = pd.read_csv('./data/202309/Flight_FIRs_Actual_20230901_20230930.csv.gz')

# --- Parse datetimes
for df in [df_f_firs_filed, df_f_firs_actual]:
    df["Entry Time"] = pd.to_datetime(df["Entry Time"], format="%d-%m-%Y %H:%M:%S")
    df["Exit Time"] = pd.to_datetime(df["Exit Time"], format="%d-%m-%Y %H:%M:%S")


In [ ]:
df_f_firs_filed[df_f_firs_filed['ECTRL ID']==265837588]

In [ ]:
df_f_firs_actual[df_f_firs_actual['ECTRL ID']==265837588]

In [ ]:
# Example of computing difference between plan and actual on crossing the FIRS

# --- Select a specific ECTRL ID
ectrl_id = 265837588
filed_sel = df_f_firs_filed[df_f_firs_filed["ECTRL ID"] == ectrl_id]
actual_sel = df_f_firs_actual[df_f_firs_actual["ECTRL ID"] == ectrl_id]


# --- Merge filed and actual by FIR ID (and maybe Sequence Number)
merged = pd.merge(
    filed_sel,
    actual_sel,
    on=["Sequence Number", "FIR ID"],
    suffixes=("_filed", "_actual")
)

# --- Compute entry and exit time differences (in minutes)
merged["Entry Δ (min)"] = (merged["Entry Time_actual"] - merged["Entry Time_filed"]).dt.total_seconds() / 60
merged["Exit Δ (min)"] = (merged["Exit Time_actual"] - merged["Exit Time_filed"]).dt.total_seconds() / 60
merged["Extra time in FIR"] = merged["Exit Δ (min)"] - merged["Entry Δ (min)"]

# --- Compute average timing shift per ECTRL ID (optional)
avg_shift = merged.groupby("Sequence Number")[["Entry Δ (min)", "Exit Δ (min)"]].mean()

# --- Display for one flight
print(merged[["Sequence Number", "FIR ID", "Entry Δ (min)", "Exit Δ (min)", "Extra time in FIR"]])



### AUAs
#### ATC Unit Airspaces

In [ ]:
# Flight AUAs
df_f_auas = pd.read_csv('./data/202309/Flight_AUAs_Filed_20230901_20230930.csv.gz')
df_a_auas = pd.read_csv('./data/202309/Flight_AUAs_Actual_20230901_20230930.csv.gz')
df_f_auas.head()

In [ ]:
len(df_f_auas)

In [ ]:
# CTA control area --> Usually above Control Zones (CTR)
# UTA upper control area (higher airspace)
# TMA terminal manouver area (around airport(s))
# CTR Control Zone (from ground (around airport))
df_f_auas[df_f_auas['ECTRL ID']==265835879]#265837588]

In [ ]:
# List of TMAs
df_f_auas[df_f_auas['AUA ID'].str.endswith('TMA')]['AUA ID'].drop_duplicates()

In [ ]:
df_f_auas[df_f_auas['AUA ID']=='EGTTICTA']